In [ ]:
아래와 같은 순서로 크롤링/엑셀 데이터를 취합하여 Interest rate prediction.csv생성
1. 연준 기준 금리(Fed benchmark interest rate)(컬럼-DFEDTARU:기준금리상한,DFEDTARL:기준금리하한)
2. 연준 기준 금리 예측치(FOMC 회의별 정책금리 전망)(컬럼:IR_FORCAST)
3. 실질 국내총생산(Real Gross Domestic Product)(컬럼:IR_FORCAST)
4. 실업률(Unemployment Rate)(컬럼:UNRATE)
5. 개인 소비 지출: 체인형 가격 지수(Personal Consumption Expenditures:Chain-type Price Index)(컬럼:PCEPI)
6. 명목이자율(미국 국채 10년물(^TNX)))(컬럼:TNX)
7. USD환율(USD)		
8. 한은 기준 금리(KR_IR)
9. ECB 기준 금리(ECB_IR)

In [3]:
pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas_datareader as web
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
# 종료일자 선언(2024-08-01 최종 데이터를 만들어 하루씩 더 빼줌)
end1 = datetime.datetime.now() - datetime.timedelta(days=1) # 날짜데이터 타입인 경우(20240801인 경우 20240731일까지 조회)
end2 = datetime.datetime.now() - datetime.timedelta(days=2) # 날짜데이터 타입인 경우(20240801인 경우 20240730일까지 조회)

1. 연준 기준 금리(Fed benchmark interest rate)-상한/하한으로 표기
(휴일자 포함)

In [6]:
start = datetime.datetime(1900, 7, 19) # 시작일자

# Fetching Federal Funds Rate data from FRED
df_fed_funds_target_rate_upper = web.DataReader('DFEDTARU', 'fred', start, end1)
df_fed_funds_target_rate_upper = df_fed_funds_target_rate_upper.reset_index()
print(df_fed_funds_target_rate_upper.head())
df_fed_funds_target_rate_lower = web.DataReader('DFEDTARL', 'fred', start, end1)
df_fed_funds_target_rate_lower = df_fed_funds_target_rate_lower.reset_index()
print(df_fed_funds_target_rate_lower.tail())

        DATE  DFEDTARU
0 2008-12-16      0.25
1 2008-12-17      0.25
2 2008-12-18      0.25
3 2008-12-19      0.25
4 2008-12-20      0.25
           DATE  DFEDTARL
5702 2024-07-27      5.25
5703 2024-07-28      5.25
5704 2024-07-29      5.25
5705 2024-07-30      5.25
5706 2024-07-31      5.25


In [7]:
df_fed_funds_target_rate_upper.dtypes

DATE        datetime64[ns]
DFEDTARU           float64
dtype: object

In [10]:
df_fed_funds_target_rate_lower.dtypes

DATE        datetime64[ns]
DFEDTARL           float64
dtype: object

2. FOMC 회의별 정책금리 전망을 보여주는 서비스로, 대표적인 것
블룸버그의 WIRP 화면, CME의 FedWatch Tool=>과거 데이터 수집은 유료이고 범위 내 확률(예)5-5.25 10%,5.25-5.5-90%)이라  아래 자료 사용하여 엑셀에 저장
https://www.investing.com/economic-calendar/interest-rate-decision-168
(데이터 가공 필요)

In [13]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [14]:
import os

# 현재 작업 디렉토리 확인
print("Current working directory:", os.getcwd())

Current working directory: C:\python code\multicampus\3. semiproject\Federal Funds Target Range Forcast\Fed1\crawlling_program


In [17]:
file_path = './../../Fed1_dataset/original_data/연준(Fed) 금리 결정.xlsx'

# FED_ir의 의미 : FED interest rate
df_FED_IR = pd.read_excel(file_path, engine='openpyxl')

df_FED_IR.head()

,발표일,시간,실제,예측,이전
0,1982-09-28,03:00:00,0.1025,NaN,NaN
1,1982-10-02,03:00:00,0.1000,NaN,0.1025
2,1982-10-08,03:00:00,0.0950,NaN,0.1000
3,1982-11-20,03:00:00,0.0900,NaN,0.0950
4,1982-12-15,03:00:00,0.0850,NaN,0.0900


In [19]:
file_path = './../../Fed1_dataset/original_data/연준(Fed) 금리 결정.xlsx'

# FED_ir의 의미 : FED interest rate
df_FED_IR = pd.read_excel(file_path, engine='openpyxl')

percent_columns = ['실제', '예측', '이전']
for col in percent_columns:
    # 율의 경우 100을 곱한 값을 사용
    df_FED_IR[col] = df_FED_IR[col].astype('float') * 100

df_FED_IR.iloc[70:150]

,발표일,시간,실제,예측,이전
70,1989-06-07,03:00:00,9.56,NaN,9.81
71,1989-07-08,03:00:00,9.31,NaN,9.56
72,1989-07-28,03:00:00,9.06,NaN,9.31
73,1989-10-20,03:00:00,8.75,NaN,9.06
74,1989-11-07,03:00:00,8.50,NaN,8.75
...,...,...,...,...,...
145,2007-11-01,03:00:00,4.50,NaN,4.75
146,2007-12-12,03:00:00,4.25,NaN,4.50
147,2008-01-23,03:15:00,3.50,NaN,4.25
148,2008-01-31,03:15:00,3.00,NaN,3.50


In [21]:
# CSV 파일 읽기
file_path = './../../Fed1_dataset/original_data/연준(Fed) 금리 결정.xlsx'
df_FED_IR = pd.read_excel(file_path, engine='openpyxl')
df_FED_IR.head()

percent_columns = ['실제', '예측', '이전']
for col in percent_columns:
    # 율의 경우 100을 곱한 값을 사용
    df_FED_IR[col] = df_FED_IR[col].astype('float') * 100

# 전체 날짜 범위 생생 : end2까지 생성
new_dates = pd.date_range(start='1982-09-28', end=end1, freq='D')

# 문자열 형식으로 변환
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_FED_IR_new = pd.DataFrame(new_dates_str, columns=['DATE'])
df_FED_IR_new['IR_FORCAST'] = np.nan

# for 루프를 사용하여 값을 채우기 (이전의 NaN이 아닌 가장 가까운 이전 값으로 채우기)
for i in range(len(df_FED_IR_new)):
    current_date = df_FED_IR_new.loc[i, 'DATE']
    mask = df_FED_IR['발표일'] <= current_date
    if mask.any():
        closest_date_value = df_FED_IR.loc[mask, '예측'].values[-1]
        df_FED_IR_new.loc[i, 'IR_FORCAST'] = closest_date_value

# NaN 값을 직전 값으로 채우기
df_FED_IR_new['IR_FORCAST'].fillna(method='ffill', inplace=True)

# 결과 출력
print(df_FED_IR_new.iloc[9430:9470])  # 예시 구간 출력(20080806 전후 2%)
print(df_FED_IR_new.iloc[9480:9520])  # 예시 구간 출력(20081008 전후 2%)
print(len(df_FED_IR_new))
print(df_FED_IR_new.iloc[13670:13730])  # 예시 구간 출력(20200304,20200306 전후 1.75%,20200430 2%)
print(df_FED_IR_new.tail())

            DATE  IR_FORCAST
9430  2008-07-23         2.0
9431  2008-07-24         2.0
9432  2008-07-25         2.0
9433  2008-07-26         2.0
9434  2008-07-27         2.0
9435  2008-07-28         2.0
9436  2008-07-29         2.0
9437  2008-07-30         2.0
9438  2008-07-31         2.0
9439  2008-08-01         2.0
9440  2008-08-02         2.0
9441  2008-08-03         2.0
9442  2008-08-04         2.0
9443  2008-08-05         2.0
9444  2008-08-06         2.0
9445  2008-08-07         2.0
9446  2008-08-08         2.0
9447  2008-08-09         2.0
9448  2008-08-10         2.0
9449  2008-08-11         2.0
9450  2008-08-12         2.0
9451  2008-08-13         2.0
9452  2008-08-14         2.0
9453  2008-08-15         2.0
9454  2008-08-16         2.0
9455  2008-08-17         2.0
9456  2008-08-18         2.0
9457  2008-08-19         2.0
9458  2008-08-20         2.0
9459  2008-08-21         2.0
9460  2008-08-22         2.0
9461  2008-08-23         2.0
9462  2008-08-24         2.0
9463  2008-08-

C:\Users\dybae\AppData\Local\Temp\ipykernel_25604\2054871841.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_FED_IR_new['IR_FORCAST'].fillna(method='ffill', inplace=True)
C:\Users\dybae\AppData\Local\Temp\ipykernel_25604\2054871841.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_FED_IR_new['IR_FORCAST'].fillna(method='ffill', inplace=True)


In [23]:
df_FED_IR.dtypes

발표일    datetime64[ns]
시간             object
실제            float64
예측            float64
이전            float64
dtype: object

In [25]:
# GDP성장률
import pandas as pd

file_path = './../../Fed1_dataset/original_data/A191RL1Q225SBEA.csv'

df_GDP = pd.read_csv(file_path)

print(df_GDP.head())
print(df_GDP.tail())

         DATE  A191RL1Q225SBEA
0  1947-04-01             -1.0
1  1947-07-01             -0.8
2  1947-10-01              6.4
3  1948-01-01              6.2
4  1948-04-01              6.8
           DATE  A191RL1Q225SBEA
304  2023-04-01              2.1
305  2023-07-01              4.9
306  2023-10-01              3.4
307  2024-01-01              1.4
308  2024-04-01              2.8


In [27]:
df_GDP.dtypes

DATE                object
A191RL1Q225SBEA    float64
dtype: object

In [67]:
# GDP성장전망을 시계열 데이터로 변환
file_path = './../../Fed1_dataset/original_data/A191RL1Q225SBEA.csv'

# Read the Excel file
df_GDP = pd.read_csv(file_path)

df_GDP.head()

# 전체 날짜 범위 생생 : end2까지 생성
new_dates = pd.date_range(start='1947-04-02', end=end1, freq='D')

# 문자열 형식으로 변환
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_GDP_new = pd.DataFrame(new_dates_str, columns=['DATE'])

# 특정 범위에 따라 값 채우기(명칭은 GDPGR로)
df_GDP_new['GDPGR'] = np.nan

# for 루프를 사용하여 값을 채우기
for i in range(len(df_GDP_new)):
    current_date = df_GDP_new.loc[i, 'DATE']
    # 해당 날짜보다 작거나 같은 가장 가까운 날짜의 값을 찾음
    mask = df_GDP['DATE'] <= current_date
    if mask.any():
        closest_date_value = df_GDP.loc[mask, 'A191RL1Q225SBEA'].values[-1]
        df_GDP_new.loc[i, 'GDPGR'] = closest_date_value

# 결과 출력
print(df_GDP_new.iloc[80:100])
print(df_GDP_new.iloc[28121:28141])
print(df_GDP_new.tail())

          DATE  GDPGR
80  1947-06-21   -1.0
81  1947-06-22   -1.0
82  1947-06-23   -1.0
83  1947-06-24   -1.0
84  1947-06-25   -1.0
85  1947-06-26   -1.0
86  1947-06-27   -1.0
87  1947-06-28   -1.0
88  1947-06-29   -1.0
89  1947-06-30   -1.0
90  1947-07-01   -0.8
91  1947-07-02   -0.8
92  1947-07-03   -0.8
93  1947-07-04   -0.8
94  1947-07-05   -0.8
95  1947-07-06   -0.8
96  1947-07-07   -0.8
97  1947-07-08   -0.8
98  1947-07-09   -0.8
99  1947-07-10   -0.8
             DATE  GDPGR
28121  2024-03-29    1.4
28122  2024-03-30    1.4
28123  2024-03-31    1.4
28124  2024-04-01    2.8
28125  2024-04-02    2.8
28126  2024-04-03    2.8
28127  2024-04-04    2.8
28128  2024-04-05    2.8
28129  2024-04-06    2.8
28130  2024-04-07    2.8
28131  2024-04-08    2.8
28132  2024-04-09    2.8
28133  2024-04-10    2.8
28134  2024-04-11    2.8
28135  2024-04-12    2.8
28136  2024-04-13    2.8
28137  2024-04-14    2.8
28138  2024-04-15    2.8
28139  2024-04-16    2.8
28140  2024-04-17    2.8
             

In [19]:
df_GDP_new.dtypes

DATE      object
GDPGR    float64
dtype: object

In [29]:
# 실업률 전망
import pandas as pd

file_path = './../../Fed1_dataset/original_data/UNRATE.csv'

df_UNRATE = pd.read_csv(file_path)

print(df_UNRATE.head())
print(df_UNRATE.tail())

         DATE  UNRATE
0  1948-01-01     3.4
1  1948-02-01     3.8
2  1948-03-01     4.0
3  1948-04-01     3.9
4  1948-05-01     3.5
           DATE  UNRATE
913  2024-02-01     3.9
914  2024-03-01     3.8
915  2024-04-01     3.9
916  2024-05-01     4.0
917  2024-06-01     4.1


In [33]:
# 실업률 전망을 시계열 데이터로 변환
file_path = './../../Fed1_dataset/original_data/UNRATE.csv'

df_UNRATE = pd.read_csv(file_path)

df_UNRATE.head()

# 전체 날짜 범위 생생 : end2까지 생성
new_dates = pd.date_range(start='1948-01-01', end=end1, freq='D')

# 문자열 형식으로 변환
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_UNRATE_new = pd.DataFrame(new_dates_str, columns=['DATE'])
print(len(df_UNRATE_new))
# 특정 범위에 따라 값 채우기
df_UNRATE_new['UNRATE'] = np.nan

# for 루프를 사용하여 값을 채우기
for i in range(len(df_UNRATE_new)):
    current_date = df_UNRATE_new.loc[i, 'DATE']
    # 해당 날짜보다 작거나 같은 가장 가까운 날짜의 값을 찾음
    mask = df_UNRATE['DATE'] <= current_date
    if mask.any():
        closest_date_value = df_UNRATE.loc[mask, 'UNRATE'].values[-1]
        df_UNRATE_new.loc[i, 'UNRATE'] = closest_date_value

# 결과 출력
print(df_UNRATE_new.iloc[80:100])
print(len(df_UNRATE_new))
print(df_UNRATE_new.iloc[27900:27920])
print(df_UNRATE_new.tail())

27972
          DATE  UNRATE
80  1948-03-21     4.0
81  1948-03-22     4.0
82  1948-03-23     4.0
83  1948-03-24     4.0
84  1948-03-25     4.0
85  1948-03-26     4.0
86  1948-03-27     4.0
87  1948-03-28     4.0
88  1948-03-29     4.0
89  1948-03-30     4.0
90  1948-03-31     4.0
91  1948-04-01     3.9
92  1948-04-02     3.9
93  1948-04-03     3.9
94  1948-04-04     3.9
95  1948-04-05     3.9
96  1948-04-06     3.9
97  1948-04-07     3.9
98  1948-04-08     3.9
99  1948-04-09     3.9
27972
             DATE  UNRATE
27900  2024-05-21     4.0
27901  2024-05-22     4.0
27902  2024-05-23     4.0
27903  2024-05-24     4.0
27904  2024-05-25     4.0
27905  2024-05-26     4.0
27906  2024-05-27     4.0
27907  2024-05-28     4.0
27908  2024-05-29     4.0
27909  2024-05-30     4.0
27910  2024-05-31     4.0
27911  2024-06-01     4.1
27912  2024-06-02     4.1
27913  2024-06-03     4.1
27914  2024-06-04     4.1
27915  2024-06-05     4.1
27916  2024-06-06     4.1
27917  2024-06-07     4.1
27918  2024

In [25]:
df_UNRATE_new.dtypes

DATE       object
UNRATE    float64
dtype: object

In [35]:
# 물가(인플레이션) 전망(PCE inflation)
import pandas as pd

file_path = './../../Fed1_dataset/original_data/PCEPI.csv'

df_PCEPI = pd.read_csv(file_path)

print(df_PCEPI.head())
print(df_PCEPI.tail())

         DATE   PCEPI
0  1959-01-01  15.164
1  1959-02-01  15.179
2  1959-03-01  15.189
3  1959-04-01  15.219
4  1959-05-01  15.227
           DATE    PCEPI
781  2024-02-01  122.367
782  2024-03-01  122.782
783  2024-04-01  123.109
784  2024-05-01  123.146
785  2024-06-01  123.243


In [37]:
# 물가(인플레이션) 전망(PCE inflation)을 시계열 데이터로 변환
file_path = './../../Fed1_dataset/original_data/PCEPI.csv'

df_PCEPI = pd.read_csv(file_path)

df_PCEPI.head()

# 전체 날짜 범위 생성
new_dates = pd.date_range(start='1959-01-01', end=end1, freq='D')
print(len(new_dates))
# 문자열 형식으로 변환
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_PCEPI_new = pd.DataFrame(new_dates_str, columns=['DATE'])
print(len(df_PCEPI_new))
# 특정 범위에 따라 값 채우기
df_PCEPI_new['PCEPI'] = np.nan

# for 루프를 사용하여 값을 채우기
for i in range(len(df_PCEPI_new)):
    current_date = df_PCEPI_new.loc[i, 'DATE']
    # 해당 날짜보다 작거나 같은 가장 가까운 날짜의 값을 찾음
    mask = df_PCEPI['DATE'] <= current_date
    if mask.any():
        closest_date_value = df_PCEPI.loc[mask, 'PCEPI'].values[-1]
        df_PCEPI_new.loc[i, 'PCEPI'] = closest_date_value

# 결과 출력
print(df_PCEPI_new.iloc[80:100])
print(len(df_PCEPI_new))
print(df_PCEPI_new.iloc[23850:23870])
print(df_PCEPI_new.tail())


23954
23954
          DATE   PCEPI
80  1959-03-22  15.189
81  1959-03-23  15.189
82  1959-03-24  15.189
83  1959-03-25  15.189
84  1959-03-26  15.189
85  1959-03-27  15.189
86  1959-03-28  15.189
87  1959-03-29  15.189
88  1959-03-30  15.189
89  1959-03-31  15.189
90  1959-04-01  15.219
91  1959-04-02  15.219
92  1959-04-03  15.219
93  1959-04-04  15.219
94  1959-04-05  15.219
95  1959-04-06  15.219
96  1959-04-07  15.219
97  1959-04-08  15.219
98  1959-04-09  15.219
99  1959-04-10  15.219
23954
             DATE    PCEPI
23850  2024-04-19  123.109
23851  2024-04-20  123.109
23852  2024-04-21  123.109
23853  2024-04-22  123.109
23854  2024-04-23  123.109
23855  2024-04-24  123.109
23856  2024-04-25  123.109
23857  2024-04-26  123.109
23858  2024-04-27  123.109
23859  2024-04-28  123.109
23860  2024-04-29  123.109
23861  2024-04-30  123.109
23862  2024-05-01  123.146
23863  2024-05-02  123.146
23864  2024-05-03  123.146
23865  2024-05-04  123.146
23866  2024-05-05  123.146
23867  2024-0

In [28]:
df_PCEPI_new.dtypes

DATE      object
PCEPI    float64
dtype: object

In [39]:
# 미국 국채 10년물 조회(CBOE Interest Rate 10 Year T No)
df_TNX = yf.download('^TNX', '1962-01-02',end1)
df_TNX.head()
df_TNX.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-25,4.206,4.266,4.196,4.256,4.256,0
2024-07-26,4.245,4.252,4.192,4.200,4.200,0
2024-07-29,4.163,4.186,4.151,4.178,4.178,0
2024-07-30,4.174,4.184,4.130,4.143,4.143,0
2024-07-31,4.128,4.134,4.091,4.109,4.109,0


In [41]:
# 미국 국채 10년물 조회(CBOE Interest Rate 10 Year T No) =>휴일 감안하여 수정종가(Adj Close)만 적용
df_TNX = yf.download('^TNX', '1962-01-02',end1)
df_TNX.head()
# yahoo-finance data의 경우 아래와 같이 추가하여 Data를 정상적으로 인식
df_TNX = df_TNX.reset_index()

# 전체 날짜 범위 생성
new_dates = pd.date_range(start='1962-01-02', end=end1, freq='D')

# 문자열 형식으로 변환
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_TNX_new = pd.DataFrame(new_dates_str, columns=['DATE'])
print(len(df_TNX_new))
# 특정 범위에 따라 값 채우기
df_TNX_new['TNX'] = np.nan

# for 루프를 사용하여 값을 채우기
for i in range(len(df_TNX_new)):
    current_date = df_TNX_new.loc[i, 'DATE']
    # 해당 날짜보다 작거나 같은 가장 가까운 날짜의 값을 찾음
    mask = df_TNX['Date'] <= current_date
    if mask.any(): # true 조건
        # 2024-07-31만 시가 (미국시간 08:44분, 한국시간 21:44분)
        if current_date == '2024-07-31' :
            closest_date_value = df_TNX.loc[mask, 'Open'].values[-1] #true조건 마지막 값 
        else :
            closest_date_value = df_TNX.loc[mask, 'Adj Close'].values[-1]
        df_TNX_new.loc[i, 'TNX'] = closest_date_value

# 결과 출력
print(df_TNX_new.iloc[0:20])
print(len(df_TNX_new))
print(df_TNX_new.tail())

[*********************100%%**********************]  1 of 1 completed


22857
          DATE   TNX
0   1962-01-02  4.06
1   1962-01-03  4.03
2   1962-01-04  3.99
3   1962-01-05  4.02
4   1962-01-06  4.02
5   1962-01-07  4.02
6   1962-01-08  4.03
7   1962-01-09  4.05
8   1962-01-10  4.07
9   1962-01-11  4.08
10  1962-01-12  4.08
11  1962-01-13  4.08
12  1962-01-14  4.08
13  1962-01-15  4.10
14  1962-01-16  4.13
15  1962-01-17  4.12
16  1962-01-18  4.11
17  1962-01-19  4.11
18  1962-01-20  4.11
19  1962-01-21  4.11
22857
             DATE    TNX
22852  2024-07-27  4.200
22853  2024-07-28  4.200
22854  2024-07-29  4.178
22855  2024-07-30  4.143
22856  2024-07-31  4.128


In [31]:
df_TNX_new.dtypes

DATE     object
TNX     float64
dtype: object

In [43]:
# USD환율
file_path = './../../Fed1_dataset/original_data/원화의 대미달러, 원화의 대위안_대엔 환율.csv'

df_USD = pd.read_csv(file_path)

print(df_USD.head())
print(df_USD.tail())

           변환     원자료
0  1990/03/02  693.00
1  1990/03/03     NaN
2  1990/03/05  695.10
3  1990/03/06  696.40
4  1990/03/07  697.60
              변환       원자료
8959  2024/07/25  1,385.40
8960  2024/07/26  1,385.80
8961  2024/07/29  1,381.90
8962  2024/07/30  1,385.30
8963  2024/07/31  1,376.50


In [45]:
import pandas as pd
import numpy as np

# CSV 파일 읽기
file_path = './../../Fed1_dataset/original_data/원화의 대미달러, 원화의 대위안_대엔 환율.csv'
df_USD = pd.read_csv(file_path)

# 날짜 형식 변경
df_USD['변환'] = pd.to_datetime(df_USD['변환'], format='%Y/%m/%d')

# 전체 날짜 범위 생성
new_dates = pd.date_range(start='1990-03-02', end=end1, freq='D')
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_USD_new = pd.DataFrame(new_dates_str, columns=['DATE'])
df_USD_new['DATE'] = pd.to_datetime(df_USD_new['DATE'], format='%Y-%m-%d')
df_USD_new['USD'] = np.nan

# for 루프를 사용하여 값을 채우기 (이전의 NaN이 아닌 가장 가까운 이전 값으로 채우기)
for i in range(len(df_USD_new)):
    current_date = df_USD_new.loc[i, 'DATE']
    mask = df_USD['변환'] <= current_date
    if mask.any():
        closest_date_value = df_USD.loc[mask, '원자료'].values[-1]
        if isinstance(closest_date_value, str): # str형인 경우
            closest_date_value = closest_date_value.replace(',', '')  # 쉼표 제거
        df_USD_new.loc[i, 'USD'] = float(closest_date_value)  # 값을 float으로 변환

# NaN 값을 직전 값으로 채우기
#df_USD_new['USD'].fillna(method='ffill', inplace=True)
df_USD_new['USD'] = df_USD_new['USD'].ffill()
# 결과 출력
print(df_USD_new.iloc[0:50])  # 예시 구간 출력
print(len(df_USD_new))
print(df_USD_new.iloc[9190:9210])  # 예시 구간 출력

         DATE    USD
0  1990-03-02  693.0
1  1990-03-03  693.0
2  1990-03-04  693.0
3  1990-03-05  695.1
4  1990-03-06  696.4
5  1990-03-07  697.6
6  1990-03-08  698.1
7  1990-03-09  698.6
8  1990-03-10  698.6
9  1990-03-11  698.6
10 1990-03-12  698.8
11 1990-03-13  698.6
12 1990-03-14  697.9
13 1990-03-15  696.5
14 1990-03-16  696.4
15 1990-03-17  696.4
16 1990-03-18  696.4
17 1990-03-19  697.0
18 1990-03-20  697.6
19 1990-03-21  697.7
20 1990-03-22  698.5
21 1990-03-23  699.9
22 1990-03-24  699.9
23 1990-03-25  699.9
24 1990-03-26  701.0
25 1990-03-27  700.9
26 1990-03-28  700.3
27 1990-03-29  701.8
28 1990-03-30  702.3
29 1990-03-31  702.3
30 1990-04-01  702.3
31 1990-04-02  704.4
32 1990-04-03  705.8
33 1990-04-04  706.3
34 1990-04-05  706.3
35 1990-04-06  706.7
36 1990-04-07  706.7
37 1990-04-08  706.7
38 1990-04-09  707.0
39 1990-04-10  707.0
40 1990-04-11  705.6
41 1990-04-12  706.4
42 1990-04-13  706.3
43 1990-04-14  706.3
44 1990-04-15  706.3
45 1990-04-16  707.0
46 1990-04-17

In [47]:
# 한은 기준 금리
file_path = './../../Fed1_dataset/original_data/한국은행 기준금리 및 여수신금리.csv'

df_KR_IR = pd.read_csv(file_path)

print(df_KR_IR.head())
print(df_KR_IR.tail())

           변환  원자료
0  1994/01/03  NaN
1  1994/01/04  NaN
2  1994/01/05  NaN
3  1994/01/06  NaN
4  1994/01/07  NaN
              변환  원자료
7820  2024/07/25  3.5
7821  2024/07/26  3.5
7822  2024/07/29  3.5
7823  2024/07/30  3.5
7824  2024/07/31  3.5


In [57]:
# KR_IR
file_path = './../../Fed1_dataset/original_data/한국은행 기준금리 및 여수신금리.csv'

df_KR_IR = pd.read_csv(file_path)

# 날짜 형식이 달라 아래와 같이 변경
df_KR_IR['변환'] = pd.to_datetime(df_KR_IR['변환'], format='%Y/%m/%d')

# 전체 날짜 범위 생성
new_dates = pd.date_range(start='1994-01-03', end=end2, freq='D')
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_KR_IR_new = pd.DataFrame(new_dates_str, columns=['DATE'])
df_KR_IR_new['DATE'] = pd.to_datetime(df_KR_IR_new['DATE'], format='%Y-%m-%d')
df_KR_IR_new['KR_IR'] = np.nan

# for 루프를 사용하여 값을 채우기
for i in range(len(df_KR_IR_new)):
    current_date = df_KR_IR_new.loc[i, 'DATE']
    mask = df_KR_IR['변환'] <= current_date
    if mask.any():
        closest_date_value = df_KR_IR.loc[mask, '원자료'].values[-1]
        df_KR_IR_new.loc[i, 'KR_IR'] = closest_date_value

# 결과 출력 2020-09-23
print(df_KR_IR_new.iloc[1900:1950])  # 예시 구간 출력
print(len(df_KR_IR_new))
print(df_KR_IR_new.iloc[9750:9800])  # 예시 구간 출력
df_KR_IR.tail()

           DATE  KR_IR
1900 1999-03-18    NaN
1901 1999-03-19    NaN
1902 1999-03-20    NaN
1903 1999-03-21    NaN
1904 1999-03-22    NaN
1905 1999-03-23    NaN
1906 1999-03-24    NaN
1907 1999-03-25    NaN
1908 1999-03-26    NaN
1909 1999-03-27    NaN
1910 1999-03-28    NaN
1911 1999-03-29    NaN
1912 1999-03-30    NaN
1913 1999-03-31    NaN
1914 1999-04-01    NaN
1915 1999-04-02    NaN
1916 1999-04-03    NaN
1917 1999-04-04    NaN
1918 1999-04-05    NaN
1919 1999-04-06    NaN
1920 1999-04-07    NaN
1921 1999-04-08    NaN
1922 1999-04-09    NaN
1923 1999-04-10    NaN
1924 1999-04-11    NaN
1925 1999-04-12    NaN
1926 1999-04-13    NaN
1927 1999-04-14    NaN
1928 1999-04-15    NaN
1929 1999-04-16    NaN
1930 1999-04-17    NaN
1931 1999-04-18    NaN
1932 1999-04-19    NaN
1933 1999-04-20    NaN
1934 1999-04-21    NaN
1935 1999-04-22    NaN
1936 1999-04-23    NaN
1937 1999-04-24    NaN
1938 1999-04-25    NaN
1939 1999-04-26    NaN
1940 1999-04-27    NaN
1941 1999-04-28    NaN
1942 1999-0

,변환,원자료
7820,2024-07-25,3.5
7821,2024-07-26,3.5
7822,2024-07-29,3.5
7823,2024-07-30,3.5
7824,2024-07-31,3.5


In [51]:
# ECB
file_path = './../../Fed1_dataset/original_data/ECB Data Portal.csv'

df_ECB = pd.read_csv(file_path)

print(df_ECB.head())
print(df_ECB.tail())

         DATE  TIME PERIOD  \
0  1999-01-01  01 Jan 1999   
1  1999-01-04  04 Jan 1999   
2  1999-01-22  22 Jan 1999   
3  1999-04-09  09 Apr 1999   
4  1999-11-05  05 Nov 1999   

   Main refinancing operations - fixed rate tenders (fixed rate) (date of changes) - Level (FM.B.U2.EUR.4F.KR.MRR_FR.LEV)  
0                                                3.0                                                                       
1                                                3.0                                                                       
2                                                3.0                                                                       
3                                                2.5                                                                       
4                                                3.0                                                                       
          DATE  TIME PERIOD  \
34  2023-05-10  10 May 2023   
35  2023-06-2

In [59]:
# ECB_IR
file_path = './../../Fed1_dataset/original_data/ECB Data Portal.csv'

df_ECB_IR = pd.read_csv(file_path)

# 전체 날짜 범위 생성
new_dates = pd.date_range(start='1999-01-01', end=end1, freq='D')

# 문자열 형식으로 변환
new_dates_str = new_dates.strftime('%Y-%m-%d')

# 데이터프레임 생성
df_ECB_IR_new = pd.DataFrame(new_dates_str, columns=['DATE'])
print(len(df_ECB_IR_new))
# 특정 범위에 따라 값 채우기
df_ECB_IR_new['ECB_IR'] = np.nan

# for 루프를 사용하여 값을 채우기
for i in range(len(df_ECB_IR_new)):
    current_date = df_ECB_IR_new.loc[i, 'DATE']
    # 해당 날짜보다 작거나 같은 가장 가까운 날짜의 값을 찾음
    mask = df_ECB_IR['DATE'] <= current_date
    if mask.any():
        closest_date_value = df_ECB_IR.loc[mask, 'Main refinancing operations - fixed rate tenders (fixed rate) (date of changes) - Level (FM.B.U2.EUR.4F.KR.MRR_FR.LEV)'].values[-1]
        df_ECB_IR_new.loc[i, 'ECB_IR'] = closest_date_value

#df_ECB_IR_new = df_ECB_IR_new.drop(columns=['TIME PERIOD'])
# 결과 출력
print(df_ECB_IR_new.iloc[0:20])
print(len(df_ECB_IR_new))
print(df_ECB_IR_new.iloc[3830:3850])
print(df_ECB_IR_new.tail())

9344
          DATE  ECB_IR
0   1999-01-01     3.0
1   1999-01-02     3.0
2   1999-01-03     3.0
3   1999-01-04     3.0
4   1999-01-05     3.0
5   1999-01-06     3.0
6   1999-01-07     3.0
7   1999-01-08     3.0
8   1999-01-09     3.0
9   1999-01-10     3.0
10  1999-01-11     3.0
11  1999-01-12     3.0
12  1999-01-13     3.0
13  1999-01-14     3.0
14  1999-01-15     3.0
15  1999-01-16     3.0
16  1999-01-17     3.0
17  1999-01-18     3.0
18  1999-01-19     3.0
19  1999-01-20     3.0
9344
            DATE  ECB_IR
3830  2009-06-27     1.0
3831  2009-06-28     1.0
3832  2009-06-29     1.0
3833  2009-06-30     1.0
3834  2009-07-01     1.0
3835  2009-07-02     1.0
3836  2009-07-03     1.0
3837  2009-07-04     1.0
3838  2009-07-05     1.0
3839  2009-07-06     1.0
3840  2009-07-07     1.0
3841  2009-07-08     1.0
3842  2009-07-09     1.0
3843  2009-07-10     1.0
3844  2009-07-11     1.0
3845  2009-07-12     1.0
3846  2009-07-13     1.0
3847  2009-07-14     1.0
3848  2009-07-15     1.0
3849  2

데이터 MERGE
1. MERGE이전 Date의 Data type을 object로 통일
2. 일자 컬럼명이 다른 것은 DATE로 통인 

In [61]:
df_fed_funds_target_rate_upper['DATE'] = df_fed_funds_target_rate_upper['DATE'].astype(str)
df_fed_funds_target_rate_lower['DATE'] = df_fed_funds_target_rate_lower['DATE'].astype(str)
df_USD_new['DATE']   = df_USD_new['DATE'].astype(str) 
df_KR_IR_new['DATE'] = df_KR_IR_new['DATE'].astype(str)
df_ECB_IR_new['DATE']= df_ECB_IR_new['DATE'].astype(str)
#df_fed_funds_target_rate_upper['DATE'] = df_fed_funds_target_rate_upper['DATE'].astype(str)

In [63]:
# Date=>DATE로 변경
df_TNX_new = df_TNX_new.rename(columns={'Date': 'DATE'})

In [45]:
# 데이터 MERGE sample
import pandas as pd

# 예제 DataFrame 생성
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'value1': ['A', 'B', 'C']
})

df2 = pd.DataFrame({
    'id': [2, 3, 4],
    'value2': ['D', 'E', 'F']
})

df3 = pd.DataFrame({
    'id': [1, 3, 4],
    'value3': ['G', 'H', 'I']
})

# df1과 df2를 'id'를 기준으로 병합
merged_df = pd.merge(df1, df2, on='id', how='inner')

# 병합된 데이터프레임에 df3를 'id'를 기준으로 병합
merged_df = pd.merge(merged_df, df3, on='id', how='inner')

# id가 3인 행을 필터링
df4 = merged_df[merged_df['id'] == 3]

print(df4)

   id value1 value2 value3
0   3      C      E      H


In [69]:
# 데이터 프레임음 Interest rate prediction의 약자

df_ir_pred = pd.merge(df_fed_funds_target_rate_upper, df_fed_funds_target_rate_lower, on='DATE', how='inner') # 미국 기준 금리상한+하단
print(df_ir_pred)
df_ir_pred = pd.merge(df_ir_pred, df_FED_IR_new, on='DATE', how='inner')  # 연준 기준 금리 예측치
df_ir_pred = pd.merge(df_ir_pred, df_GDP_new, on='DATE', how='inner')     # GDP
df_ir_pred = pd.merge(df_ir_pred, df_UNRATE_new, on='DATE', how='inner')  # 실업률
df_ir_pred = pd.merge(df_ir_pred, df_PCEPI_new, on='DATE', how='inner')   # 개인 소비 지출: 체인형 가격 지수
df_ir_pred = pd.merge(df_ir_pred, df_TNX_new, on='DATE', how='inner')     # 미국 국채 10년 금리
df_ir_pred = pd.merge(df_ir_pred, df_USD_new, on='DATE', how='inner')     # USD환율
df_ir_pred = pd.merge(df_ir_pred, df_KR_IR_new, on='DATE', how='inner')   # 한국기준 금리
df_ir_pred = pd.merge(df_ir_pred, df_ECB_IR_new, on='DATE', how='inner')  # 유럽기준 금리

            DATE  DFEDTARU  DFEDTARL
0     2008-12-16      0.25      0.00
1     2008-12-17      0.25      0.00
2     2008-12-18      0.25      0.00
3     2008-12-19      0.25      0.00
4     2008-12-20      0.25      0.00
...          ...       ...       ...
5702  2024-07-27      5.50      5.25
5703  2024-07-28      5.50      5.25
5704  2024-07-29      5.50      5.25
5705  2024-07-30      5.50      5.25
5706  2024-07-31      5.50      5.25

[5707 rows x 3 columns]


In [71]:
df_ir_pred.iloc[10:20] # 2009-01-01 데이터 검증

,DATE,DFEDTARU,DFEDTARL,IR_FORCAST,GDPGR,UNRATE,PCEPI,TNX,USD,KR_IR,ECB_IR
10,2008-12-26,0.25,0.0,0.5,-8.5,7.3,88.098,2.137,1299.0,3.0,2.5
11,2008-12-27,0.25,0.0,0.5,-8.5,7.3,88.098,2.137,1299.0,3.0,2.5
12,2008-12-28,0.25,0.0,0.5,-8.5,7.3,88.098,2.137,1299.0,3.0,2.5
13,2008-12-29,0.25,0.0,0.5,-8.5,7.3,88.098,2.096,1263.0,3.0,2.5
14,2008-12-30,0.25,0.0,0.5,-8.5,7.3,88.098,2.087,1259.5,3.0,2.5
15,2008-12-31,0.25,0.0,0.5,-8.5,7.3,88.098,2.244,1259.5,3.0,2.5
16,2009-01-01,0.25,0.0,0.5,-4.5,7.8,88.108,2.244,1259.5,3.0,2.5
17,2009-01-02,0.25,0.0,0.5,-4.5,7.8,88.108,2.416,1321.0,3.0,2.5
18,2009-01-03,0.25,0.0,0.5,-4.5,7.8,88.108,2.416,1321.0,3.0,2.5
19,2009-01-04,0.25,0.0,0.5,-4.5,7.8,88.108,2.416,1321.0,3.0,2.5


In [73]:
df_ir_pred.iloc[40:60] # 2009-02-01 데이터 검증

,DATE,DFEDTARU,DFEDTARL,IR_FORCAST,GDPGR,UNRATE,PCEPI,TNX,USD,KR_IR,ECB_IR
40,2009-01-25,0.25,0.0,0.50,-4.5,7.8,88.108,2.622,1390.9,2.5,2.0
41,2009-01-26,0.25,0.0,0.50,-4.5,7.8,88.108,2.643,1390.9,2.5,2.0
42,2009-01-27,0.25,0.0,0.50,-4.5,7.8,88.108,2.519,1390.9,2.5,2.0
43,2009-01-28,0.25,0.0,0.50,-4.5,7.8,88.108,2.656,1376.1,2.5,2.0
44,2009-01-29,0.25,0.0,0.25,-4.5,7.8,88.108,2.815,1378.5,2.5,2.0
45,2009-01-30,0.25,0.0,0.25,-4.5,7.8,88.108,2.844,1379.5,2.5,2.0
46,2009-01-31,0.25,0.0,0.25,-4.5,7.8,88.108,2.844,1379.5,2.5,2.0
47,2009-02-01,0.25,0.0,0.25,-4.5,8.3,88.266,2.844,1379.5,2.5,2.0
48,2009-02-02,0.25,0.0,0.25,-4.5,8.3,88.266,2.719,1390.0,2.5,2.0
49,2009-02-03,0.25,0.0,0.25,-4.5,8.3,88.266,2.842,1389.5,2.5,2.0


In [97]:
# 'DATE' 열을 datetime 형식으로 변환
df_ir_pred['DATE'] = pd.to_datetime(df_ir_pred['DATE'])

# 'DATE' 열을 인덱스로 설정
df_ir_pred.set_index('DATE', inplace=True)

# 매월 첫째 날 기준으로 리샘플링
df_ir_pred_monthly = df_ir_pred.resample('MS').first().dropna() 
# 매월 마지막 날 기준으로 리샘플링
df_ir_pred_monthly = df_ir_pred.resample('M').last().dropna()

# df_ir_pred의 index를 reset
df_ir_pred.reset_index(inplace=True)
# 2024-07-31일 변경전 코드
df_ir_pred['DATE'] = df_ir_pred['DATE'].astype(str)
#print(df_ir_pred_monthly.dtypes)
df_ir_pred_monthly.reset_index(inplace=True)
df_ir_pred_monthly['DATE'] = df_ir_pred_monthly['DATE'].dt.to_period('M').dt.to_timestamp()
#df_ir_pred_monthly['DATE'] 
df_ir_pred_monthly['DATE'] = df_ir_pred_monthly['DATE'].astype(str)

# df_ir_pred의 최초일자가 2008-12-16이므로 2008-12-01은 제외하도록 함
#df_ir_pred_monthly = df_ir_pred_monthly[df_ir_pred_monthly['DATE'] != '2008-12-01']
# 결과 출력
print(df_ir_pred_monthly.head())
print(df_ir_pred_monthly.tail())

         DATE  DFEDTARU  DFEDTARL  IR_FORCAST  GDPGR  UNRATE   PCEPI    TNX  \
0  2008-12-01      0.25       0.0        0.50   -8.5     7.3  88.098  2.244   
1  2009-01-01      0.25       0.0        0.25   -4.5     7.8  88.108  2.844   
2  2009-02-01      0.25       0.0        0.25   -4.5     8.3  88.266  3.041   
3  2009-03-01      0.25       0.0        0.25   -4.5     8.7  88.169  2.685   
4  2009-04-01      0.25       0.0        0.25   -0.7     9.0  88.295  3.124   

      USD  KR_IR  ECB_IR  
0  1259.5    3.0    2.50  
1  1379.5    2.5    2.00  
2  1534.0    2.0    2.00  
3  1383.5    2.0    1.50  
4  1282.0    2.0    1.25  
           DATE  DFEDTARU  DFEDTARL  IR_FORCAST  GDPGR  UNRATE    PCEPI  \
183  2024-03-01       5.5      5.25         5.5    1.4     3.8  122.782   
184  2024-04-01       5.5      5.25         5.5    2.8     3.9  123.109   
185  2024-05-01       5.5      5.25         5.5    2.8     4.0  123.146   
186  2024-06-01       5.5      5.25         5.5    2.8     4.1 

C:\Users\dybae\AppData\Local\Temp\ipykernel_25604\291982717.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_ir_pred_monthly = df_ir_pred.resample('M').last().dropna()


In [99]:
df_ir_pred_monthly.dtypes

DATE           object
DFEDTARU      float64
DFEDTARL      float64
IR_FORCAST    float64
GDPGR         float64
UNRATE        float64
PCEPI         float64
TNX           float64
USD           float64
KR_IR         float64
ECB_IR        float64
dtype: object

CSV전환 확인=>index=False로 index 출력되지 않게 함)

In [105]:
df_ir_pred_monthly.to_csv("./../../Fed1_dataset/dummy_data/data/Interest rate prediction.csv",index=False)